In [ ]:

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import json
import os
from collections import Counter
import numpy as np
from sklearn.metrics import classification_report, accuracy_score
import copy
import warnings


warnings.filterwarnings("ignore", category=UserWarning)


class Config:
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    BATCH_SIZE = 32
    EMBEDDING_DIM = 100
    HIDDEN_DIM = 128
    NUM_EPOCHS = 10
    LEARNING_RATE = 0.001
    PAD_TOKEN = "<pad>"
    UNK_TOKEN = "<unk>"

print(f"Using device: {Config.DEVICE}")


Using device: cpu


In [ ]:

class ScratchLSTMCell(nn.Module):
    """A single step of a Long Short-Term Memory network."""
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.W = nn.Linear(input_size + hidden_size, 4 * hidden_size, bias=True)

    def forward(self, x, states):
        h_prev, c_prev = states
        combined = torch.cat([x, h_prev], dim=1)
        gates = self.W(combined)
        
        
        i, f, g, o = torch.chunk(gates, 4, dim=1)
        
        c_next = torch.sigmoid(f) * c_prev + torch.sigmoid(i) * torch.tanh(g)
        h_next = torch.sigmoid(o) * torch.tanh(c_next)
        return h_next, c_next

class ScratchLSTM(nn.Module):
    """A multi-step LSTM layer that processes a sequence."""
    def __init__(self, input_size, hidden_size, bidirectional=False):
        super().__init__()
        self.hidden_size = hidden_size
        self.bidirectional = bidirectional
        self.forward_cell = ScratchLSTMCell(input_size, hidden_size)
        if self.bidirectional:
            self.backward_cell = ScratchLSTMCell(input_size, hidden_size)

    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        h0 = torch.zeros(batch_size, self.hidden_size).to(x.device)
        c0 = torch.zeros(batch_size, self.hidden_size).to(x.device)
        
        # Forward pass
        outputs_fwd = []
        h_t, c_t = (h0, c0)
        for t in range(seq_len):
            h_t, c_t = self.forward_cell(x[:, t, :], (h_t, c_t))
            outputs_fwd.append(h_t)
        final_h_fwd, final_c_fwd = h_t, c_t
        outputs_fwd = torch.stack(outputs_fwd, dim=1)

        if not self.bidirectional:
            return outputs_fwd, (final_h_fwd.unsqueeze(0), final_c_fwd.unsqueeze(0))

        # Backward pass
        outputs_bwd = []
        h_t, c_t = (h0, c0)
        for t in reversed(range(seq_len)):
            h_t, c_t = self.backward_cell(x[:, t, :], (h_t, c_t))
            outputs_bwd.insert(0, h_t)
        final_h_bwd, final_c_bwd = h_t, c_t
        outputs_bwd = torch.stack(outputs_bwd, dim=1)

        outputs = torch.cat([outputs_fwd, outputs_bwd], dim=2)
        final_h = torch.stack([final_h_fwd, final_h_bwd], dim=0)
        final_c = torch.stack([final_c_fwd, final_c_bwd], dim=0)
        return outputs, (final_h, final_c)

print("Scratch LSTM implementation is defined.")

Scratch LSTM implementation is defined.


In [ ]:

def load_actual_slurp_data_fixed():
    """Load and preprocess ACTUAL SLURP dataset with WORKING span-based entities"""
    print("Loading ACTUAL SLURP dataset with working span alignment...")
    
    def read_slurp_file(file_path):
        data = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data.append(json.loads(line.strip()))
        return data
    
    
    try:
        train_data = read_slurp_file("slurp/train.jsonl")
        test_data = read_slurp_file("slurp/test.jsonl")
        print(f"ACTUAL SLURP Dataset: {len(train_data)} train, {len(test_data)} test examples")
    except Exception as e:
        print(f"Error loading SLURP data: {e}")
        raise
    
  
    word_counter = Counter()
    intent_labels = set()
    slot_labels = set(['O'])
    
    print("Building vocabulary from ACTUAL SLURP data...")
    for split_data in [train_data, test_data]:
        for item in split_data:
            sentence = item['sentence']
            tokens = sentence.split()
            word_counter.update([token.lower() for token in tokens])
            
            scenario = item['scenario']
            action = item['action']
            intent = f"{scenario}_{action}"
            intent_labels.add(intent)
            
            
            entities = item.get('entities', [])
            for entity in entities:
                slot_type = entity.get('type', 'unknown')
                slot_labels.add(f"B-{slot_type}")
                slot_labels.add(f"I-{slot_type}")
   
    word_vocab = {Config.PAD_TOKEN: 0, Config.UNK_TOKEN: 1}
    for i, word in enumerate(word_counter, start=2):
        word_vocab[word] = i
    
    intent_vocab = {intent: i for i, intent in enumerate(intent_labels)}
    slot_vocab = {slot: i for i, slot in enumerate(slot_labels)}
    
   
    idx_to_intent = {i: intent for intent, i in intent_vocab.items()}
    idx_to_slot = {i: slot for slot, i in slot_vocab.items()}
    
    vocabs = {
        "word": word_vocab, 
        "intent": intent_vocab, 
        "slot": slot_vocab,
        "idx_to_intent": idx_to_intent,
        "idx_to_slot": idx_to_slot
    }
    
    print(f"ACTUAL SLURP Vocab: {len(word_vocab)} words, {len(intent_vocab)} intents, {len(slot_vocab)} slots")
    
    return train_data, test_data, vocabs

class FixedActualSLURPDataset(Dataset):
    def __init__(self, data, vocabs):
        self.data = data
        self.vocabs = vocabs
        
    def __len__(self): 
        return len(self.data)
    
    def __getitem__(self, idx):
        item = self.data[idx]
        
        sentence = item['sentence']
        raw_tokens = sentence.split()
        
        
        tokens = []
        for token in raw_tokens:
            token_idx = self.vocabs['word'].get(token.lower(), self.vocabs['word'][Config.UNK_TOKEN])
            tokens.append(token_idx)
        
        
        scenario = item['scenario']
        action = item['action']
        intent = f"{scenario}_{action}"
        intent_label = self.vocabs['intent'][intent]
        
        
        slot_labels = [self.vocabs['slot']['O']] * len(raw_tokens)
        
        entities = item.get('entities', [])
        
        
        char_to_token = {}
        current_char = 0
        
        for token_idx, token in enumerate(raw_tokens):
           
            for _ in range(len(token)):
                char_to_token[current_char] = token_idx
                current_char += 1
            
            current_char += 1
        
        for entity in entities:
            if not isinstance(entity, dict):
                continue
                
            entity_type = entity.get('type', 'unknown')
            spans = entity.get('span', [])
            
            if not spans:
                continue
            
            
            entity_token_indices = set()
            for char_pos in spans:
                if char_pos in char_to_token:
                    entity_token_indices.add(char_to_token[char_pos])
            
            entity_token_indices = sorted(entity_token_indices)
            
            if entity_token_indices:
                
                b_tag = f"B-{entity_type}"
                i_tag = f"I-{entity_type}"
                
                if b_tag in self.vocabs['slot'] and i_tag in self.vocabs['slot']:
                   
                    slot_labels[entity_token_indices[0]] = self.vocabs['slot'][b_tag]
                   
                    for j in range(1, len(entity_token_indices)):
                        slot_labels[entity_token_indices[j]] = self.vocabs['slot'][i_tag]
        
        return {
            "tokens": torch.LongTensor(tokens), 
            "intent": intent_label, 
            "slots": torch.LongTensor(slot_labels)
        }


In [ ]:

class IntentClassifier(nn.Module):
    def __init__(self, vocab_size, intent_size, emb_dim, hidden_dim):
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size + 10, emb_dim, padding_idx=0) 
        self.recurrent_layer = ScratchLSTM(emb_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, intent_size)
        
    def forward(self, tokens):
        
        tokens = torch.clamp(tokens, 0, self.embedding.num_embeddings - 1)
        embedded = self.embedding(tokens)
        _, (hidden, _) = self.recurrent_layer(embedded)
        hidden = torch.cat((hidden[0], hidden[1]), dim=1)
        return self.fc(hidden)

class SlotFiller(nn.Module):
    def __init__(self, vocab_size, slot_size, emb_dim, hidden_dim):
        super().__init__()
       
        self.embedding = nn.Embedding(vocab_size + 10, emb_dim, padding_idx=0)  
        self.recurrent_layer = ScratchLSTM(emb_dim, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, slot_size)
        
    def forward(self, tokens):
        
        tokens = torch.clamp(tokens, 0, self.embedding.num_embeddings - 1)
        embedded = self.embedding(tokens)
        recurrent_out, _ = self.recurrent_layer(embedded)
        return self.fc(recurrent_out)

class IntentClassifierWithSlots(nn.Module):
    def __init__(self, vocab_size, intent_size, slot_size, emb_dim, hidden_dim):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size + 10, emb_dim, padding_idx=0)
        self.slot_embedding = nn.Embedding(slot_size, emb_dim // 2, padding_idx=0)
        self.recurrent_layer = ScratchLSTM(emb_dim + emb_dim // 2, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, intent_size)
        
    def forward(self, tokens, predicted_slots):
        tokens = torch.clamp(tokens, 0, self.word_embedding.num_embeddings - 1)
        predicted_slots = torch.clamp(predicted_slots, 0, self.slot_embedding.num_embeddings - 1)
        
        word_embedded = self.word_embedding(tokens)
        slot_embedded = self.slot_embedding(predicted_slots)
        combined_embedded = torch.cat((word_embedded, slot_embedded), dim=2)
        _, (hidden, _) = self.recurrent_layer(combined_embedded)
        hidden = torch.cat((hidden[0], hidden[1]), dim=1)
        return self.fc(hidden)

class SlotFillerWithIntent(nn.Module):
    def __init__(self, vocab_size, intent_size, slot_size, emb_dim, hidden_dim):
        super().__init__()
        self.word_embedding = nn.Embedding(vocab_size + 10, emb_dim, padding_idx=0)
        self.intent_embedding = nn.Embedding(intent_size, emb_dim // 4)
        self.recurrent_layer = ScratchLSTM(emb_dim + emb_dim // 4, hidden_dim, bidirectional=True)
        self.fc = nn.Linear(hidden_dim * 2, slot_size)
        
    def forward(self, tokens, predicted_intent):
        tokens = torch.clamp(tokens, 0, self.word_embedding.num_embeddings - 1)
        predicted_intent = torch.clamp(predicted_intent, 0, self.intent_embedding.num_embeddings - 1)
        
        word_embedded = self.word_embedding(tokens)
        intent_embedded = self.intent_embedding(predicted_intent).unsqueeze(1)
        intent_embedded = intent_embedded.expand(-1, tokens.shape[1], -1)
        combined_embedded = torch.cat((word_embedded, intent_embedded), dim=2)
        recurrent_out, _ = self.recurrent_layer(combined_embedded)
        return self.fc(recurrent_out)

class JointModel(nn.Module):
    def __init__(self, vocab_size, intent_size, slot_size, emb_dim, hidden_dim):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 10, emb_dim, padding_idx=0)
        self.recurrent_layer = ScratchLSTM(emb_dim, hidden_dim, bidirectional=True)
        self.intent_fc = nn.Linear(hidden_dim * 2, intent_size)
        self.slot_fc = nn.Linear(hidden_dim * 2, slot_size)
        
    def forward(self, tokens):
        tokens = torch.clamp(tokens, 0, self.embedding.num_embeddings - 1)
        embedded = self.embedding(tokens)
        recurrent_out, (hidden, _) = self.recurrent_layer(embedded)
        final_hidden = torch.cat((hidden[0], hidden[1]), dim=1)
        return self.intent_fc(final_hidden), self.slot_fc(recurrent_out)

print("All five LSTM model architectures are defined.")


All five LSTM model architectures are defined.


In [ ]:

def evaluate_fixed(model, loader, model_type, best_slot_model=None, best_intent_model=None):
    model.eval()
    results = {}
    
    with torch.no_grad():
        if model_type == "intent":
            all_preds, all_true = [], []
            for batch in loader:
                outputs = model(batch['tokens'].to(Config.DEVICE))
                all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                all_true.extend(batch['intents'].numpy())
            
            if all_true:
                report = classification_report(all_true, all_preds, output_dict=True, zero_division=0)
                results['intent_accuracy'] = accuracy_score(all_true, all_preds)
                results['intent_f1_macro'] = report['macro avg']['f1-score']
                results['intent_f1_weighted'] = report['weighted avg']['f1-score']
        
        elif model_type == "slot_to_intent":
            all_preds, all_true = [], []
            for batch in loader:
                tokens = batch['tokens'].to(Config.DEVICE)
                predicted_slots = torch.argmax(best_slot_model(tokens), dim=2)
                outputs = model(tokens, predicted_slots)
                all_preds.extend(torch.argmax(outputs, dim=1).cpu().numpy())
                all_true.extend(batch['intents'].numpy())
            
            if all_true:
                report = classification_report(all_true, all_preds, output_dict=True, zero_division=0)
                results['intent_accuracy'] = accuracy_score(all_true, all_preds)
                results['intent_f1_macro'] = report['macro avg']['f1-score']
                results['intent_f1_weighted'] = report['weighted avg']['f1-score']

        elif model_type == "slot" or model_type == "intent_to_slot":
            all_preds, all_true = [], []
            slot_distribution = Counter()
            non_o_predictions = 0
            total_predictions = 0
            
            for batch in loader:
                tokens = batch['tokens'].to(Config.DEVICE)
                slots_true_np = batch['slots'].numpy()
                
                if model_type == "intent_to_slot":
                    predicted_intent = torch.argmax(best_intent_model(tokens), dim=1)
                    outputs = model(tokens, predicted_intent)
                else:
                    outputs = model(tokens)
                    
                slot_preds = torch.argmax(outputs, dim=2).cpu().numpy()
                
                for i in range(slots_true_np.shape[0]):
                    for j in range(slots_true_np.shape[1]):
                        if slots_true_np[i, j] != 0:  
                            all_preds.append(slot_preds[i, j])
                            all_true.append(slots_true_np[i, j])
                            slot_distribution[slot_preds[i, j]] += 1
                            total_predictions += 1
                            if slot_preds[i, j] != 0:  
                                non_o_predictions += 1
            
            if all_true:
            
                print(f"      Slot Prediction Distribution:")
                for slot_idx, count in slot_distribution.most_common(10):
                    slot_name = loader.dataset.vocabs['idx_to_slot'].get(slot_idx, f"UNK_{slot_idx}")
                    percentage = (count / total_predictions) * 100
                    print(f"        {slot_name}: {count} ({percentage:.1f}%)")
                print(f"      Non-O predictions: {non_o_predictions}/{total_predictions} ({non_o_predictions/total_predictions*100:.1f}%)")
                
                report = classification_report(all_true, all_preds, output_dict=True, zero_division=0)
                results['slot_accuracy'] = accuracy_score(all_true, all_preds)
                results['slot_f1_macro'] = report['macro avg']['f1-score']
                results['slot_f1_weighted'] = report['weighted avg']['f1-score']
        
        elif model_type == "joint":
            intent_preds, intent_true, slot_preds, slot_true = [], [], [], []
            slot_distribution = Counter()
            non_o_predictions = 0
            total_predictions = 0
            
            for batch in loader:
                tokens = batch['tokens'].to(Config.DEVICE)
                intents_np, slots_np = batch['intents'].numpy(), batch['slots'].numpy()
                
                intent_logits, slot_logits = model(tokens)
                intent_preds.extend(torch.argmax(intent_logits, dim=1).cpu().numpy())
                intent_true.extend(intents_np)
                
                slot_preds_batch = torch.argmax(slot_logits, dim=2).cpu().numpy()
                for i in range(slots_np.shape[0]):
                    for j in range(slots_np.shape[1]):
                        if slots_np[i, j] != 0:  
                            slot_preds.append(slot_preds_batch[i, j])
                            slot_true.append(slots_np[i, j])
                            slot_distribution[slot_preds_batch[i, j]] += 1
                            total_predictions += 1
                            if slot_preds_batch[i, j] != 0:
                                non_o_predictions += 1
            
            if intent_true:
                intent_report = classification_report(intent_true, intent_preds, output_dict=True, zero_division=0)
                results['intent_accuracy'] = accuracy_score(intent_true, intent_preds)
                results['intent_f1_macro'] = intent_report['macro avg']['f1-score']
                results['intent_f1_weighted'] = intent_report['weighted avg']['f1-score']
            
            if slot_true:
                
                print(f"      Joint Model Slot Distribution:")
                for slot_idx, count in slot_distribution.most_common(10):
                    slot_name = loader.dataset.vocabs['idx_to_slot'].get(slot_idx, f"UNK_{slot_idx}")
                    percentage = (count / total_predictions) * 100
                    print(f"        {slot_name}: {count} ({percentage:.1f}%)")
                print(f"      Non-O predictions: {non_o_predictions}/{total_predictions} ({non_o_predictions/total_predictions*100:.1f}%)")
                
                slot_report = classification_report(slot_true, slot_preds, output_dict=True, zero_division=0)
                results['slot_accuracy'] = accuracy_score(slot_true, slot_preds)
                results['slot_f1_macro'] = slot_report['macro avg']['f1-score']
                results['slot_f1_weighted'] = slot_report['weighted avg']['f1-score']
    
    return results


In [ ]:

def run_all_experiments_working_slots(train_loader, test_loader, vocabs, dataset_name):
    print(f"\n" + "#"*70)
    print(f"##   RUNNING ALL EXPERIMENTS FOR: LSTM - {dataset_name}   ##")
    print("#"*70 + "\n")

    vocab_size = len(vocabs['word'])
    intent_size = len(vocabs['intent'])
    slot_size = len(vocabs['slot'])
    
    print(f"Model Parameters: vocab_size={vocab_size}, intent_size={intent_size}, slot_size={slot_size}")
    
    
    print("Analyzing ACTUAL slot distribution from dataset...")
    actual_slot_counter = Counter()
    total_slots = 0
    non_o_slots = 0
    
    for batch in train_loader:
        slots = batch['slots']
        for i in range(slots.shape[0]):
            for j in range(slots.shape[1]):
                slot_idx = slots[i, j].item()
                if slot_idx != 0: 
                    actual_slot_counter[slot_idx] += 1
                    total_slots += 1
                    if slot_idx != vocabs['slot']['O']:
                        non_o_slots += 1
    
    print("Top 10 ACTUAL slot types in training data:")
    for slot_idx, count in actual_slot_counter.most_common(10):
        slot_name = vocabs['idx_to_slot'].get(slot_idx, f"UNK_{slot_idx}")
        percentage = (count / total_slots) * 100
        print(f"  {slot_name}: {count} ({percentage:.1f}%)")
    
    o_count = actual_slot_counter.get(vocabs['slot']['O'], 0)
    print(f"O vs non-O distribution: O={o_count} ({o_count/total_slots*100:.1f}%), non-O={non_o_slots} ({non_o_slots/total_slots*100:.1f}%)")
    
    loss_fn_intent = nn.CrossEntropyLoss()
    
    slot_weights = torch.ones(slot_size).to(Config.DEVICE)
    for slot_idx, count in actual_slot_counter.items():
        if count > 0:
            
            slot_weights[slot_idx] = total_slots / (len(actual_slot_counter) * count)
    
    print("Using class weights for slot filling to handle imbalance")
    loss_fn_slot = nn.CrossEntropyLoss(weight=slot_weights, ignore_index=0)
    
    all_results = {}

   
    print("\n EXPERIMENT 1: Independent Models")
    intent_model = IntentClassifier(vocab_size, intent_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM).to(Config.DEVICE)
    slot_model = SlotFiller(vocab_size, slot_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM).to(Config.DEVICE)
    
   
    print("  Training Intent Model...")
    optimizer = torch.optim.Adam(intent_model.parameters(), lr=Config.LEARNING_RATE)
    for epoch in range(Config.NUM_EPOCHS):
        intent_model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            tokens = batch['tokens'].to(Config.DEVICE)
            intents = batch['intents'].to(Config.DEVICE)
            
            outputs = intent_model(tokens)
            loss = loss_fn_intent(outputs, intents)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if (epoch + 1) % 2 == 0:
            print(f"    [Intent] Epoch {epoch+1}/{Config.NUM_EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")
    
    
    print("  Training Slot Model...")
    optimizer = torch.optim.Adam(slot_model.parameters(), lr=Config.LEARNING_RATE)
    
    for epoch in range(Config.NUM_EPOCHS):
        slot_model.train()
        total_loss = 0
        batch_count = 0
        
        for batch_idx, batch in enumerate(train_loader):
            optimizer.zero_grad()
            tokens = batch['tokens'].to(Config.DEVICE)
            slots = batch['slots'].to(Config.DEVICE)
            
            outputs = slot_model(tokens)
            
            
            batch_size, seq_len = slots.shape
            outputs_flat = outputs.contiguous().view(batch_size * seq_len, -1)
            slots_flat = slots.contiguous().view(-1)
            
            loss = loss_fn_slot(outputs_flat, slots_flat)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            batch_count += 1
            
        avg_loss = total_loss / batch_count
        
       
        print(f"    [Slot]   Epoch {epoch+1}/{Config.NUM_EPOCHS}, Loss: {avg_loss:.6f}")
        
        if (epoch + 1) % 3 == 0 or epoch + 1 == Config.NUM_EPOCHS:
            slot_model.eval()
            with torch.no_grad():
                sample_batch = next(iter(train_loader))
                sample_tokens = sample_batch['tokens'][:1].to(Config.DEVICE)
                sample_slots = sample_batch['slots'][:1]
                sample_outputs = slot_model(sample_tokens)
                sample_preds = torch.argmax(sample_outputs, dim=2).cpu()
                
                g
                idx_to_word = {idx: word for word, idx in vocabs['word'].items()}
                
                
                sample_token_indices = sample_tokens[0].cpu()
                sample_words = []
                for idx in sample_token_indices:
                    if idx == 0:  
                        break
                    sample_words.append(idx_to_word.get(idx.item(), f"UNK_{idx.item()}"))
                
                print(f"      Sample prediction:")
                print(f"        Text: {' '.join(sample_words)}")
                
                
                true_slots = []
                pred_slots = []
                for i in range(min(len(sample_words), sample_slots.shape[1])):
                    true_idx = sample_slots[0, i].item()
                    pred_idx = sample_preds[0, i].item()
                    if true_idx != 0: 
                        true_slots.append(vocabs['idx_to_slot'].get(true_idx, f"UNK_{true_idx}"))
                        pred_slots.append(vocabs['idx_to_slot'].get(pred_idx, f"UNK_{pred_idx}"))
                
                print(f"        True:  {true_slots}")
                print(f"        Pred:  {pred_slots}")
                
                
                unique_preds = set(pred_slots)
                o_count = pred_slots.count('O')
                total_count = len(pred_slots)
                if total_count > 0:
                    o_percentage = (o_count / total_count) * 100
                    print(f"        Unique predictions: {unique_preds}")
                    print(f"        'O' predictions: {o_count}/{total_count} ({o_percentage:.1f}%)")
            
            slot_model.train()
    
    
    best_intent_model = copy.deepcopy(intent_model)
    best_slot_model = copy.deepcopy(slot_model)
    
    print("  Evaluating Independent Models...")
    intent_results = evaluate_fixed(intent_model, test_loader, "intent")
    slot_results = evaluate_fixed(slot_model, test_loader, "slot")
    all_results['Independent'] = {**intent_results, **slot_results}

    
    print("\n🚀 EXPERIMENT 2: Slot -> Intent Pipeline")
    pipeline_model = IntentClassifierWithSlots(vocab_size, intent_size, slot_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM).to(Config.DEVICE)
    optimizer = torch.optim.Adam(pipeline_model.parameters(), lr=Config.LEARNING_RATE)
    
    for epoch in range(Config.NUM_EPOCHS):
        pipeline_model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            tokens = batch['tokens'].to(Config.DEVICE)
            intents = batch['intents'].to(Config.DEVICE)
            
            with torch.no_grad():
                predicted_slots = torch.argmax(best_slot_model(tokens), dim=2)
            
            outputs = pipeline_model(tokens, predicted_slots)
            loss = loss_fn_intent(outputs, intents)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if (epoch + 1) % 2 == 0:
            print(f"    [S->I]   Epoch {epoch+1}/{Config.NUM_EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")
    
    all_results['Slot -> Intent'] = evaluate_fixed(pipeline_model, test_loader, "slot_to_intent", best_slot_model=best_slot_model)

    
    print("\n🚀 EXPERIMENT 3: Intent -> Slot Pipeline")
    pipeline_model = SlotFillerWithIntent(vocab_size, intent_size, slot_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM).to(Config.DEVICE)
    optimizer = torch.optim.Adam(pipeline_model.parameters(), lr=Config.LEARNING_RATE)
    
    for epoch in range(Config.NUM_EPOCHS):
        pipeline_model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            tokens = batch['tokens'].to(Config.DEVICE)
            slots = batch['slots'].to(Config.DEVICE)
            
            with torch.no_grad():
                predicted_intent = torch.argmax(best_intent_model(tokens), dim=1)
            
            outputs = pipeline_model(tokens, predicted_intent)
            
            batch_size, seq_len = slots.shape
            outputs_flat = outputs.contiguous().view(batch_size * seq_len, -1)
            slots_flat = slots.contiguous().view(-1)
            
            loss = loss_fn_slot(outputs_flat, slots_flat)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if (epoch + 1) % 2 == 0:
            print(f"    [I->S]   Epoch {epoch+1}/{Config.NUM_EPOCHS}, Loss: {total_loss/len(train_loader):.6f}")
    
    all_results['Intent -> Slot'] = evaluate_fixed(pipeline_model, test_loader, "intent_to_slot", best_intent_model=best_intent_model)

    
    print("\n🚀 EXPERIMENT 4: Joint Model (Multi-Task)")
    joint_model = JointModel(vocab_size, intent_size, slot_size, Config.EMBEDDING_DIM, Config.HIDDEN_DIM).to(Config.DEVICE)
    optimizer = torch.optim.Adam(joint_model.parameters(), lr=Config.LEARNING_RATE)
    
    for epoch in range(Config.NUM_EPOCHS):
        joint_model.train()
        total_loss = 0
        for batch in train_loader:
            optimizer.zero_grad()
            tokens = batch['tokens'].to(Config.DEVICE)
            intents = batch['intents'].to(Config.DEVICE)
            slots = batch['slots'].to(Config.DEVICE)
            
            intent_logits, slot_logits = joint_model(tokens)
            
            loss_intent = loss_fn_intent(intent_logits, intents)
            
            batch_size, seq_len = slots.shape
            slot_logits_flat = slot_logits.contiguous().view(batch_size * seq_len, -1)
            slots_flat = slots.contiguous().view(-1)
            loss_slot = loss_fn_slot(slot_logits_flat, slots_flat)
            
            loss = loss_intent + loss_slot
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            
        if (epoch + 1) % 2 == 0:
            print(f"    [Joint]  Epoch {epoch+1}/{Config.NUM_EPOCHS}, Loss: {total_loss/len(train_loader):.4f}")
    
    all_results['Joint'] = evaluate_fixed(joint_model, test_loader, "joint")

    
    print_results_table(all_results, dataset_name)
    return all_results

print("Updated experiment runner for working slot data is defined.")

Updated experiment runner for working slot data is defined.


In [ ]:

if __name__ == "__main__":
    print("="*80)
    print("STARTING ACTUAL SLURP DATASET EXPERIMENTS (FIXED SPAN ALIGNMENT)")
    print("="*80)
    
    try:
        
        slurp_train_data, slurp_test_data, slurp_vocabs = load_actual_slurp_data_fixed()
        
       
        slurp_train_dataset = FixedActualSLURPDataset(slurp_train_data, slurp_vocabs)
        slurp_test_dataset = FixedActualSLURPDataset(slurp_test_data, slurp_vocabs)
        
        
        slurp_train_loader = DataLoader(slurp_train_dataset, batch_size=Config.BATCH_SIZE, shuffle=True, 
                                       collate_fn=collate_fn_factory(slurp_vocabs))
        slurp_test_loader = DataLoader(slurp_test_dataset, batch_size=Config.BATCH_SIZE, 
                                      collate_fn=collate_fn_factory(slurp_vocabs))
        
        print(f"ACTUAL SLURP - Train batches: {len(slurp_train_loader)}, Test batches: {len(slurp_test_loader)}")
        
        
        print("\n🔍 DEBUGGING ENTITY ALIGNMENT IN ACTUAL SLURP:")
        entity_examples_found = 0
        
        for i in range(min(20, len(slurp_train_data))):
            item = slurp_train_data[i]
            entities = item.get('entities', [])
            
            if entities:
                entity_examples_found += 1
                print(f"\nExample {i+1} with entities:")
                print(f"  Sentence: '{item['sentence']}'")
                print(f"  Intent: {item['scenario']}_{item['action']}")
                
                
                for j, entity in enumerate(entities):
                    print(f"  Entity {j+1}: type='{entity.get('type')}', span={entity.get('span', [])}")
                
                
                sample_item = slurp_train_dataset[i]
                idx_to_word = {idx: word for word, idx in slurp_vocabs['word'].items()}
                tokens = [idx_to_word.get(idx, 'UNK') for idx in sample_item['tokens'].tolist() if idx != 0]
                slots = [slurp_vocabs['idx_to_slot'].get(idx, 'O') for idx in sample_item['slots'].tolist()[:len(tokens)]]
                
                print(f"  Tokens: {tokens}")
                print(f"  Slots:  {slots}")
                
                
                non_o_slots = [s for s in slots if s != 'O']
                if non_o_slots:
                    print(f"  SUCCESS: Found {len(non_o_slots)} non-O slots: {non_o_slots}")
                else:
                    print(f"   PROBLEM: All slots are 'O'")
                
                if entity_examples_found >= 3:  
                    break
        
        
        print("\n ANALYZING SLOT DISTRIBUTION FROM ACTUAL SLURP:")
        slot_counter = Counter()
        total_slots = 0
        non_o_slots = 0
        
        for batch in slurp_train_loader:
            slots = batch['slots']
            for i in range(slots.shape[0]):
                for j in range(slots.shape[1]):
                    slot_idx = slots[i, j].item()
                    if slot_idx != 0:  
                        slot_counter[slot_idx] += 1
                        total_slots += 1
                        if slot_idx != slurp_vocabs['slot']['O']:
                            non_o_slots += 1
        
        print("Top 15 slot types from ACTUAL SLURP:")
        for slot_idx, count in slot_counter.most_common(15):
            slot_name = slurp_vocabs['idx_to_slot'].get(slot_idx, f"UNK_{slot_idx}")
            percentage = (count / total_slots) * 100 if total_slots > 0 else 0
            print(f"  {slot_name}: {count} ({percentage:.1f}%)")
        
        o_count = slot_counter.get(slurp_vocabs['slot']['O'], 0)
        o_percentage = (o_count / total_slots) * 100 if total_slots > 0 else 0
        non_o_percentage = (non_o_slots / total_slots) * 100 if total_slots > 0 else 0
        
        print(f"O vs non-O distribution: O={o_count} ({o_percentage:.1f}%), non-O={non_o_slots} ({non_o_percentage:.1f}%)")
        
        
        if non_o_slots > 0 and non_o_percentage > 5:  
            print(f"\n SUCCESS: Found meaningful non-O slots ({non_o_percentage:.1f}%) in ACTUAL SLURP!")
            print(" Running all 4 LSTM experiments on ACTUAL SLURP data...")
            slurp_results = run_all_experiments_working_slots(slurp_train_loader, slurp_test_loader, slurp_vocabs, "ACTUAL SLURP")
            print("\n ACTUAL SLURP DATASET EXPERIMENTS COMPLETED SUCCESSFULLY! ✨")
        else:
            print(f"\n PROBLEM: Insufficient non-O slots ({non_o_percentage:.1f}%) in ACTUAL SLURP")
            print("The span alignment might still need adjustment.")
            
            
            print("\n🔧 ADDITIONAL SPAN DEBUGGING:")
            for i, item in enumerate(slurp_train_data[:5]):
                if item.get('entities'):
                    print(f"\nDebug Example {i+1}:")
                    print(f"  Sentence: '{item['sentence']}'")
                    print(f"  Sentence length: {len(item['sentence'])}")
                    
                    
                    print(f"  Character positions (first 50):")
                    for j in range(min(50, len(item['sentence']))):
                        print(f"    {j}: '{item['sentence'][j]}'")
                    
                    entities = item.get('entities', [])
                    for j, entity in enumerate(entities):
                        spans = entity.get('span', [])
                        print(f"  Entity {j+1}: type='{entity.get('type')}', spans={spans}")
                        if spans:
                            print(f"    Span characters: {[item['sentence'][s] if s < len(item['sentence']) else 'OUT_OF_RANGE' for s in spans]}")
                    break
        
    except Exception as e:
        print(f"Error with ACTUAL SLURP dataset: {e}")
        import traceback
        traceback.print_exc()

STARTING ACTUAL SLURP DATASET EXPERIMENTS (FIXED SPAN ALIGNMENT)
Loading ACTUAL SLURP dataset with working span alignment...
ACTUAL SLURP Dataset: 11514 train, 2974 test examples
Building vocabulary from ACTUAL SLURP data...
ACTUAL SLURP Vocab: 6069 words, 60 intents, 111 slots
ACTUAL SLURP - Train batches: 360, Test batches: 93

🔍 DEBUGGING ENTITY ALIGNMENT IN ACTUAL SLURP:

Example 2 with entities:
  Sentence: 'i need an event three days from now scheduled with amy'
  Intent: calendar_set
  Entity 1: type='date', span=[4, 5, 6, 7]
  Entity 2: type='person', span=[10]
  Tokens: ['i', 'need', 'an', 'event', 'three', 'days', 'from', 'now', 'scheduled', 'with', 'amy']
  Slots:  ['O', 'B-date', 'I-date', 'B-person', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
  ✅ SUCCESS: Found 3 non-O slots: ['B-date', 'I-date', 'B-person']

Example 7 with entities:
  Sentence: 'silent mode on for the next three hours'
  Intent: audio_volume_mute
  Entity 1: type='time', span=[5, 6, 7]
  Tokens: ['silent', 'mode'

In [9]:

def compare_all_models():
    
    print(" MODEL COMPARISON DASHBOARD")
    
   
    results = {}
    
    
    print("\n ATIS DATASET MODELS")
    
    
    if os.path.exists("saved_models/intent_model.pth"):
        try:
            model = load_model_for_inference("saved_models/intent_model.pth", "intent", atis_vocabs)
            test_results = evaluate_fixed(model, atis_test_loader, "intent")
            results['ATIS_Intent_Independent'] = test_results
            print(f" Intent Model: {test_results.get('intent_accuracy', 0):.4f} accuracy")
        except Exception as e:
            print(f" ATIS Intent Model: Error - {e}")
    
    if os.path.exists("saved_models/slot_model.pth"):
        try:
            model = load_model_for_inference("saved_models/slot_model.pth", "slot", atis_vocabs)
            test_results = evaluate_fixed(model, atis_test_loader, "slot")
            results['ATIS_Slot_Independent'] = test_results
            print(f"Slot Model: {test_results.get('slot_accuracy', 0):.4f} accuracy")
        except Exception as e:
            print(f" ATIS Slot Model: Error - {e}")
    
    if os.path.exists("saved_models/joint_model.pth"):
        try:
            model = load_model_for_inference("saved_models/joint_model.pth", "joint", atis_vocabs)
            test_results = evaluate_fixed(model, atis_test_loader, "joint")
            results['ATIS_Joint'] = test_results
            print(f"Joint Model: Intent={test_results.get('intent_accuracy', 0):.4f}, Slot={test_results.get('slot_accuracy', 0):.4f}")
        except Exception as e:
            print(f" ATIS Joint Model: Error - {e}")
    
    
    print("\n SLURP DATASET MODELS")
    
    
    if os.path.exists("saved_models/slurp_intent_classifier.pth"):
        try:
            model = load_slurp_intent_model("saved_models/slurp_intent_classifier.pth", slurp_vocabs)
            test_results = evaluate_fixed(model, slurp_test_loader, "intent")
            results['SLURP_Intent_Independent'] = test_results
            print(f" Intent Model: {test_results.get('intent_accuracy', 0):.4f} accuracy")
        except Exception as e:
            print(f" SLURP Intent Model: Error - {e}")
    
    if os.path.exists("saved_models/slurp_slot_filler.pth"):
        try:
            model = load_slurp_slot_model("saved_models/slurp_slot_filler.pth", slurp_vocabs)
            test_results = evaluate_fixed(model, slurp_test_loader, "slot")
            results['SLURP_Slot_Independent'] = test_results
            print(f" Slot Model: {test_results.get('slot_accuracy', 0):.4f} accuracy")
        except Exception as e:
            print(f" SLURP Slot Model: Error - {e}")
    
    if os.path.exists("saved_models/slurp_joint_model.pth"):
        try:
            model = load_slurp_joint_model("saved_models/slurp_joint_model.pth", slurp_vocabs)
            test_results = evaluate_fixed(model, slurp_test_loader, "joint")
            results['SLURP_Joint'] = test_results
            print(f" Joint Model: Intent={test_results.get('intent_accuracy', 0):.4f}, Slot={test_results.get('slot_accuracy', 0):.4f}")
        except Exception as e:
            print(f" SLURP Joint Model: Error - {e}")
    
    
    
    print(" PERFORMANCE COMPARISON SUMMARY")
    
    print(f"{'Model':<30} | {'Intent Acc':<10} | {'Intent F1':<10} | {'Slot Acc':<10} | {'Slot F1':<10}")
    
    
    for model_name, metrics in results.items():
        intent_acc = f"{metrics.get('intent_accuracy', 0):.4f}" if 'intent_accuracy' in metrics else "N/A"
        intent_f1 = f"{metrics.get('intent_f1_macro', 0):.4f}" if 'intent_f1_macro' in metrics else "N/A"
        slot_acc = f"{metrics.get('slot_accuracy', 0):.4f}" if 'slot_accuracy' in metrics else "N/A"
        slot_f1 = f"{metrics.get('slot_f1_macro', 0):.4f}" if 'slot_f1_macro' in metrics else "N/A"
        
        print(f"{model_name:<30} | {intent_acc:<10} | {intent_f1:<10} | {slot_acc:<10} | {slot_f1:<10}")
    
    print("=" * 100)
    
    
    print("\n KEY INSIGHTS:")
    
    
    
    atis_intent_models = {k: v for k, v in results.items() if 'ATIS' in k and 'intent_accuracy' in v}
    atis_slot_models = {k: v for k, v in results.items() if 'ATIS' in k and 'slot_accuracy' in v}
    slurp_intent_models = {k: v for k, v in results.items() if 'SLURP' in k and 'intent_accuracy' in v}
    slurp_slot_models = {k: v for k, v in results.items() if 'SLURP' in k and 'slot_accuracy' in v}
    
    if atis_intent_models:
        best_atis_intent = max(atis_intent_models.items(), key=lambda x: x[1]['intent_accuracy'])
        print(f" Best ATIS Intent Model: {best_atis_intent[0]} ({best_atis_intent[1]['intent_accuracy']:.4f})")
    
    if atis_slot_models:
        best_atis_slot = max(atis_slot_models.items(), key=lambda x: x[1]['slot_accuracy'])
        print(f" Best ATIS Slot Model: {best_atis_slot[0]} ({best_atis_slot[1]['slot_accuracy']:.4f})")
    
    if slurp_intent_models:
        best_slurp_intent = max(slurp_intent_models.items(), key=lambda x: x[1]['intent_accuracy'])
        print(f" Best SLURP Intent Model: {best_slurp_intent[0]} ({best_slurp_intent[1]['intent_accuracy']:.4f})")
    
    if slurp_slot_models:
        best_slurp_slot = max(slurp_slot_models.items(), key=lambda x: x[1]['slot_accuracy'])
        print(f" Best SLURP Slot Model: {best_slurp_slot[0]} ({best_slurp_slot[1]['slot_accuracy']:.4f})")


def show_model_performance():
    """Show performance without running evaluation (uses your experiment results)"""
    print(" MODEL PERFORMANCE SUMMARY")
    
    print("\n ATIS DATASET (Flight Booking)")
    
    print(f"{'Model':<25} | {'Intent Acc':<10} | {'Slot Acc':<10} | {'Description'}")
    
    print(f"{'Intent Model':<25} | {'0.9518':<10} | {'N/A':<10} | {'Only predicts flight intent'}")
    print(f"{'Slot Model':<25} | {'N/A':<10} | {'0.9987':<10} | {'Only predicts flight entities'}")
    print(f"{'Joint Model':<25} | {'0.9485':<10} | {'0.9992':<10} | {'Predicts both simultaneously'}")
    
    print("\n SLURP DATASET (General Commands)")
    
    print(f"{'Model':<25} | {'Intent Acc':<10} | {'Slot Acc':<10} | {'Description'}")
    
    print(f"{'Intent Model':<25} | {'0.7872':<10} | {'N/A':<10} | {'Only predicts command type'}")
    print(f"{'Slot Model':<25} | {'N/A':<10} | {'0.6699':<10} | {'Only predicts entities'}")
    print(f"{'Joint Model':<25} | {'0.7784':<10} | {'0.6369':<10} | {'Predicts both simultaneously'}")
    
   


def quick_model_test():
    
    print(" QUICK MODEL TEST")
   
    
    examples = {
        "ATIS": [
            "show me flights from boston to san francisco",
            "what is the cheapest fare to los angeles"
        ],
        "SLURP": [
            "play some jazz music",
            "set alarm for 7 am tomorrow"
        ]
    }
    
    for dataset, sentences in examples.items():
        print(f"\n {dataset} Examples:")
        print("-" * 40)
        
        for sentence in sentences:
            print(f"\n '{sentence}'")
            
            if dataset == "ATIS" and os.path.exists("saved_models/joint_model.pth"):
                try:
                    model = load_model_for_inference("saved_models/joint_model.pth", "joint", atis_vocabs)
                    intent, slots = predict_joint(sentence, model, atis_vocabs)
                    print(f"  Intent: {intent}")
                    print(f"  Slots: {[f'{token}→{slot}' for token, slot in slots if slot != 'O']}")
                except:
                    print("  Model not available")
            
            elif dataset == "SLURP" and os.path.exists("saved_models/slurp_joint_model.pth"):
                try:
                    model = load_slurp_joint_model("saved_models/slurp_joint_model.pth", slurp_vocabs)
                    intent, slots = predict_joint(sentence, model, slurp_vocabs)
                    print(f"  Intent: {intent}")
                    print(f"  Slots: {[f'{token}→{slot}' for token, slot in slots if slot != 'O']}")
                except:
                    print("   Model not available")


print("Ready to compare models! Run one of these:")
print("1. compare_all_models() - Full evaluation (needs test data)")
print("2. show_model_performance() - Quick summary")
print("3. quick_model_test() - Test with examples")

Ready to compare models! Run one of these:
1. compare_all_models() - Full evaluation (needs test data)
2. show_model_performance() - Quick summary
3. quick_model_test() - Test with examples


In [10]:

show_model_performance()


 MODEL PERFORMANCE SUMMARY

 ATIS DATASET (Flight Booking)
Model                     | Intent Acc | Slot Acc   | Description
Intent Model              | 0.9518     | N/A        | Only predicts flight intent
Slot Model                | N/A        | 0.9987     | Only predicts flight entities
Joint Model               | 0.9485     | 0.9992     | Predicts both simultaneously

 SLURP DATASET (General Commands)
Model                     | Intent Acc | Slot Acc   | Description
Intent Model              | 0.7872     | N/A        | Only predicts command type
Slot Model                | N/A        | 0.6699     | Only predicts entities
Joint Model               | 0.7784     | 0.6369     | Predicts both simultaneously
